## Training File for LLM Model BigBird
This entire section contains code for preprocessing, tokenizing, label aligining the data and fine tuning BigBird for PII data detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
! pip install -U accelerate
! pip install -U transformers
# !pip install transformers[torch]
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 6.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-

In [3]:
import os

files = os.listdir('/content/drive/My Drive')
print(files)


['Fall 2023', 'WINDOWS11.23H2_A64_20240125.iso.zip', 'SE - CSCI 6231 - Group 4', 'CSCI 6441 - DBMS', 'CSCI 6364 - ML', 'Jobs', 'Resume VedantMahangade.pdf', 'Colab Notebooks', 'ML-shared']


In [4]:
import pandas as pd
import numpy as np
from transformers import RobertaTokenizer, RobertaTokenizerFast, DataCollatorForTokenClassification, RobertaForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
from transformers import AutoTokenizer
import datasets
from datasets import Dataset, DatasetDict
from ast import literal_eval
from sklearn.model_selection import train_test_split
from functools import partial
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

In [5]:
from transformers import XLMRobertaXLForTokenClassification

In [6]:
# ds_all = pd.read_csv('../data/combined_data.csv',converters={
#     'tokens': literal_eval,
#     'labels': literal_eval,
#     'trailing_whitespace': literal_eval
# })
# ds_all

FileNotFoundError: [Errno 2] No such file or directory: '../data/combined_data.csv'

In [7]:
ds_all = pd.read_json('/content/drive/MyDrive/ML-shared/train.json')
ds_all.head()

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."


In [8]:
ds_external = pd.read_csv('/content/drive/MyDrive/ML-shared/pii_dataset.csv', converters={
    'tokens': literal_eval,
    'labels': literal_eval,
    'trailing_whitespace': literal_eval
})
ds_external.rename(columns={'text': 'full_text'}, inplace=True)
ds_external = ds_external.iloc[:, :5]
ds_external.head()

,document,full_text,tokens,trailing_whitespace,labels
0,1073d46f-2241-459b-ab01-851be8d26436,"My name is Aaliyah Popova, and I am a jeweler ...","[My, name, is, Aaliyah, Popova,, and, I, am, a...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
1,5ec717a9-17ee-48cd-9d76-30ae256c9354,"My name is Konstantin Becker, and I'm a develo...","[My, name, is, Konstantin, Becker,, and, I'm, ...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
2,353da41e-7799-4071-ab20-d959b362612e,"As Mieko Mitsubishi, an account manager at a p...","[As, Mieko, Mitsubishi,, an, account, manager,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."
3,9324ee01-7bdc-41b1-a7a5-01307f72c20d,"My name is Kazuo Sun, and I'm an air traffic c...","[My, name, is, Kazuo, Sun,, and, I'm, an, air,...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4,971fe266-2739-4f1b-979b-7f64e07d5a4a,"My name is Arina Sun, and I'm a dental hygieni...","[My, name, is, Arina, Sun,, and, I'm, a, denta...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."


In [9]:
ds_all.document = ds_all.document.astype(int)
max_doc_id = ds_all.document.max()
print("Last doc id used:", max_doc_id)
ds_external.document = pd.Series(np.arange(max_doc_id+1, max_doc_id+1+len(ds_external)))

Last doc id used: 22687


In [10]:
ds_all = pd.concat([ds_all, ds_external], axis=0)
ds_all

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
4429,27117,"Hello, I'm Nicholas Moore, a man with a rich t...","[Hello,, I'm, Nicholas, Moore,, a, man, with, ...","[True, True, True, True, True, True, True, Tru...","[O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O..."
4430,27118,"Hello, my name is Alexey Novikov and I'm a psy...","[Hello,, my, name, is, Alexey, Novikov, and, I...","[True, True, True, True, True, True, True, Tru...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
4431,27119,"My name is Ludmila Inoue, and I'm a person wit...","[My, name, is, Ludmila, Inoue,, and, I'm, a, p...","[True, True, True, True, True, True, True, Tru...","[O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O, O..."
4432,27120,"Dr. Tu Garcia, a renowned dermatologist, embar...","[Dr., Tu, Garcia,, a, renowned, dermatologist,...","[True, True, True, True, True, True, True, Tru...","[O, B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O..."


In [11]:
label_list = ['O',
                 'B-NAME_STUDENT',
                 'I-NAME_STUDENT',
                 'B-EMAIL',
                 'I-EMAIL',
                 'B-USERNAME',
                 'I-USERNAME',
                 'B-ID_NUM',
                 'I-ID_NUM' ,
                 'B-PHONE_NUM',
                 'I-PHONE_NUM',
                 'B-URL_PERSONAL',
                 'I-URL_PERSONAL',
                 'B-STREET_ADDRESS',
                 'I-STREET_ADDRESS']

encoder = {'O':0,
           'B-NAME_STUDENT': 1,
           'I-NAME_STUDENT': 2,
           'B-EMAIL': 3,
           'I-EMAIL': 4,
           'B-USERNAME': 5,
           'I-USERNAME': 6,
           'B-ID_NUM': 7,
           'I-ID_NUM' : 8,
           'B-PHONE_NUM': 9,
           'I-PHONE_NUM': 10,
           'B-URL_PERSONAL': 11,
           'I-URL_PERSONAL': 12,
           'B-STREET_ADDRESS': 13,
           'I-STREET_ADDRESS': 14}
decoder = {0: 'O',
           1: 'B-NAME_STUDENT',
           2: 'I-NAME_STUDENT',
           3: 'B-EMAIL',
           4: 'I-EMAIL',
           5: 'B-USERNAME',
           6: 'I-USERNAME',
           7: 'B-ID_NUM',
           8: 'I-ID_NUM',
           9: 'B-PHONE_NUM',
           10: 'I-PHONE_NUM',
           11: 'B-URL_PERSONAL',
           12: 'I-URL_PERSONAL',
           13: 'B-STREET_ADDRESS',
           14: 'I-STREET_ADDRESS'}

In [12]:
def create_mapped_labels(labels):
    mapped_labels = pd.DataFrame({
                        'mapped_labels': labels
                    })['mapped_labels'].map(encoder).tolist()
    return mapped_labels

In [13]:
ds_all['mapped_labels'] = ds_all['labels'].apply(create_mapped_labels)
ds_all.head(3)

,document,full_text,tokens,trailing_whitespace,labels,mapped_labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, ..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O...","[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
ds_all['is_labels'] = ds_all['labels'].apply(lambda labels: any(label != 'O' for label in labels))

ds_pos = ds_all[ds_all['is_labels']]
print(len(ds_pos), "left after filtering.")

5370 left after filtering.


In [15]:
train, valid = train_test_split(ds_pos, test_size=0.1, shuffle=True, random_state=1278)

In [16]:
train_ds = Dataset.from_dict({
        'document': [document for document in train['document']],
        'full_text': [full_text for full_text in train['full_text']],
        'tokens': [tokens for tokens in train['tokens']],
        'trailing_whitespace': [trailing_whitespace for trailing_whitespace in train['trailing_whitespace']],
        'labels': [labels for labels in train['labels']],
        'mapped_labels': [mapped_labels for mapped_labels in train['mapped_labels']]
    })

valid_ds = Dataset.from_dict({
        'document': [document for document in valid['document']],
        'full_text': [full_text for full_text in valid['full_text']],
        'tokens': [tokens for tokens in valid['tokens']],
        'trailing_whitespace': [trailing_whitespace for trailing_whitespace in valid['trailing_whitespace']],
        'labels': [labels for labels in valid['labels']],
        'mapped_labels': [mapped_labels for mapped_labels in valid['mapped_labels']]
    })

In [38]:
from transformers import BigBirdTokenizerFast, BigBirdForTokenClassification

In [54]:
# tokenizer = AutoTokenizer.from_pretrained("roberta-base")
tokenizer = BigBirdTokenizerFast.from_pretrained("google/bigbird-roberta-base",
                                                 attention_type = "block_sparse",
                                                #  num_random_blocks = 3,
                                                 block_size = 2
)

In [47]:
model = BigBirdForTokenClassification.from_pretrained('google/bigbird-roberta-base',
    num_labels=15,
    id2label=decoder,
    label2id=encoder,
    ignore_mismatched_sizes=True
)

Some weights of BigBirdForTokenClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [56]:
def tokenize_and_align_labels(row):
    # print(type(row["tokens"]), type(row["tokens"][0]))
    tokenized_inputs = tokenizer(
        # row["tokens"][0], truncation=False #, is_split_into_words=True
        row["tokens"], padding='max_length', truncation=False, is_split_into_words=True, max_length=4096
        # row["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = row["mapped_labels"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [57]:
tokenized_train = train_ds.map(
    tokenize_and_align_labels,
    batched=True
)
tokenized_valid = valid_ds.map(
    tokenize_and_align_labels,
    batched=True
)

Map:   0%|          | 0/4833 [00:00<?, ? examples/s]

Map:   0%|          | 0/537 [00:00<?, ? examples/s]

In [58]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, pad_to_multiple_of=2)

In [59]:


def compute_metrics(p, label_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    if precision == 0 and recall == 0:
        f1_score = 0
    else:
        f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)

    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results

In [60]:
training_args = TrainingArguments(
    output_dir='output',
    fp16=True,
    learning_rate=3.4e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.038,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    report_to='none'
)

In [61]:
def make_trainer(train, valid):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=valid,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=partial(compute_metrics, label_list=label_list)
    )
    return trainer

In [ ]:
trainer = make_trainer(tokenized_train, tokenized_valid)
print("Starting training...")
trainer.train()

Starting training...


Epoch,Training Loss,Validation Loss


In [ ]:
import shutil
import os
from pathlib import Path

colab_directory_path = '/content/output'

drive_destination_dir = '/content/drive/My Drive/'


zip_archive_path = Path(drive_destination_dir) / f"{Path(colab_directory_path).name}.zip"
shutil.make_archive(str(zip_archive_path.parent / zip_archive_path.stem), 'zip', str(colab_directory_path))


In [ ]:
trainer.save_model("roberta-pii_v1")
tokenizer.save_pretrained("roberta-pii-tokenizer_v1")

In [ ]:
colab_directory_path = '/content/roberta-pii-tokenizer_v1'

drive_destination_dir = '/content/drive/My Drive/'


zip_archive_path = Path(drive_destination_dir) / f"{Path(colab_directory_path).name}.zip"
shutil.make_archive(str(zip_archive_path.parent / zip_archive_path.stem), 'zip', str(colab_directory_path))

In [ ]:
colab_directory_path = '/content/roberta-pii_v1'

drive_destination_dir = '/content/drive/My Drive/'


zip_archive_path = Path(drive_destination_dir) / f"{Path(colab_directory_path).name}.zip"
shutil.make_archive(str(zip_archive_path.parent / zip_archive_path.stem), 'zip', str(colab_directory_path))